In [1]:
#Dependencies
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
#path to csv files and merge
mouse_metadata = pd.read_csv("../Resources/Mouse_metadata.csv")
study_results = pd.read_csv("../Resources/Study_results.csv")
mouse_study = pd.merge(mouse, study, how="outer", on="Mouse ID")
mouse_study.tail()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4
1892,z969,Naftisol,Male,9,30,45,73.867845,4


In [24]:
#display number of unique mice IDs
mouse_study["Mouse ID"].nunique()

249

In [44]:
#check for any mouse ID with duplicate time points
mouse_study["Mouse ID"].value_counts()

g989    13
a203    10
u327    10
g570    10
b559    10
        ..
n482     1
l872     1
d133     1
h428     1
x226     1
Name: Mouse ID, Length: 249, dtype: int64

In [50]:
#display the data associated with duplicate mouse ID
new_index = mouse_study.set_index("Mouse ID")
new_index.loc["g989", :]

,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
Mouse ID,,,,,,,
g989,Propriva,Female,21,26,0,45.000000,0
g989,Propriva,Female,21,26,0,45.000000,0
g989,Propriva,Female,21,26,5,48.786801,0
g989,Propriva,Female,21,26,5,47.570392,0
g989,Propriva,Female,21,26,10,51.745156,0
g989,Propriva,Female,21,26,10,49.880528,0
g989,Propriva,Female,21,26,15,51.325852,1
g989,Propriva,Female,21,26,15,53.442020,0
g989,Propriva,Female,21,26,20,55.326122,1


In [52]:
#create a new DataFrame where this data is removed
remove_dup = new_index.drop(["g989"])
remove_dup

,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
Mouse ID,,,,,,,
k403,Ramicane,Male,21,16,0,45.000000,0
k403,Ramicane,Male,21,16,5,38.825898,0
k403,Ramicane,Male,21,16,10,35.014271,1
k403,Ramicane,Male,21,16,15,34.223992,1
k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...
z969,Naftisol,Male,9,30,25,63.145652,2
z969,Naftisol,Male,9,30,30,65.841013,3
z969,Naftisol,Male,9,30,35,69.176246,4


In [54]:
#display new number of unique mouse IDs
clean_df = remove_dup.reset_index()
clean_df["Mouse ID"].nunique()

248